In [43]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords


from sklearn.model_selection import train_test_split

In [30]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abhishekwaghchaure/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/abhishekwaghchaure/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [31]:
df = pd.read_csv('/Users/abhishekwaghchaure/Desktop/Datasets/emails.csv')
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [32]:
print(df.info())
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   file     517401 non-null  object
 1   message  517401 non-null  object
dtypes: object(2)
memory usage: 7.9+ MB
None
(517401, 2)


In [33]:
df['message'][2]

"Message-ID: <24216240.1075855687451.JavaMail.evans@thyme>\nDate: Wed, 18 Oct 2000 03:00:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: leah.arsdall@enron.com\nSubject: Re: test\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: Leah Van Arsdall\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_Dec2000\\Notes Folders\\'sent mail\nX-Origin: Allen-P\nX-FileName: pallen.nsf\n\ntest successful.  way to go!!!"

In [34]:
## Missing Values
df.isnull().sum()

file       0
message    0
dtype: int64

#### No missing Values Detected

In [35]:
## Extracting Email Headers and body
def split_email_content(email):
    headers, body = email.split('\n\n', 1) if '\n\n' in email else (email, "")
    return pd.Series([headers, body])

In [36]:
df[['headers', 'body']] =df['message'].apply(split_email_content)

In [37]:
df.head()

,file,message,headers,body
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast\n\n
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy,\n\n Can you send me a schedule of the s..."
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.


In [42]:
# print(df['message'][2])
print(df['body'][2])
print(df['headers'][2])

test successful.  way to go!!!
Message-ID: <24216240.1075855687451.JavaMail.evans@thyme>
Date: Wed, 18 Oct 2000 03:00:00 -0700 (PDT)
From: phillip.allen@enron.com
To: leah.arsdall@enron.com
Subject: Re: test
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Leah Van Arsdall
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf


In [39]:
df = df.drop(columns = ['message'])
print(f"Email split into headers and body")

Email split into headers and body


In [44]:
## Extract Usefull Headers
def extract_header_value(header_text, field):
    match = re.search(fr"{field}: (.+)", header_text, re.IGNORECASE)
    return match.group(1).strip() if match else None


In [45]:
df['subject'] = df['headers'].apply(lambda x: extract_header_value(x, 'Subject'))
df['from'] = df['headers'].apply(lambda x: extract_header_value(x, 'from'))
df['to'] = df['headers'].apply(lambda x: extract_header_value(x, 'to'))

In [54]:
df.iloc[1000]

file                              allen-p/all_documents/462.
headers    Message-ID: <1081797.1075855696183.JavaMail.ev...
body       Lucy,\n\nHere is the rentroll.\n\nMy only ques...
subject                                                 None
from                                 phillip.allen@enron.com
to                                stagecoachmama@hotmail.com
Name: 1000, dtype: object

In [55]:
## Cleaning Text
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

In [56]:
df['clean_body'] = df['body'].apply(clean_text)

In [64]:
df['clean_body'][3]

'Randy Can you send me a schedule of the salary and level of everyone in the scheduling group Plus your thoughts on any changes that need to be made Patti S for example Phillip'

In [65]:
df['body'][3]

'Randy,\n\n Can you send me a schedule of the salary and level of everyone in the \nscheduling group.  Plus your thoughts on any changes that need to be made.  \n(Patti S for example)\n\nPhillip'

In [67]:
def preprocess_text(text):
    corpus = []
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in set(stopwords.words('english'))]
    corpus.append(words)
    return corpus

In [68]:
df['processed_body']  = df['clean_body'].apply(preprocess_text)

In [71]:
# Saving the preprocessed data
df.to_csv("/Users/abhishekwaghchaure/Desktop/Datasets/preprocessed_emails.csv", index=False)
print("\nPreprocessed dataset saved as 'preprocessed_emails.csv'.")


Preprocessed dataset saved as 'preprocessed_emails.csv'.


In [70]:
df['processed_body']

0                                        [[Here, forecast]]
1         [[Traveling, business, meeting, takes, fun, tr...
2                             [[test, successful, way, go]]
3         [[Randy, Can, send, schedule, salary, level, e...
4                            [[Lets, shoot, Tuesday, 1145]]
                                ...                        
517396    [[This, trade, OILSPECHEDGENG, John, Lavoratos...
517397    [[Some, position, Alberta, Term, book, I, send...
517398    [[2, Original, Message, From, Doucet, Dawn, Se...
517399    [[Analyst, Rank, Stephane, Brodeur, 1, Chad, C...
517400    [[think, YMCA, class, people, recovering, hear...
Name: processed_body, Length: 517401, dtype: object